In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
from metrics_features import fi_perm
from dnn_tau import Dnn_tau
from data_extractor import Data_extractor_v2, output_vars_v2
import os
import fnmatch
from utils import normalize, bucketize, split_dataset
from copy import deepcopy
import pickle
from metrics_features import *

In [6]:
path = "/data/hnl/prompt_tau/anatuple/nanoV10/TEST9/"
features = deepcopy(output_vars_v2)
features.extend(['signal_label', 'channel', 'event_type', 'mass_hyp'])
channels = os.listdir(path)
relative_path = "/anatuple/"

In [7]:
values = []
for i in range(len(features)):
    values.append([])
data = dict(zip(features, values))

for channel in channels:
    extractor = Data_extractor_v2(channel)
    data = extractor(path+channel+relative_path, data=data)

In [8]:
N = len(data['event'])
n_bkg = N-sum([data['signal_label'][i] for i in range(len(data['signal_label']))])
data_norm = normalize(pd.DataFrame(data), 'mass_hyp', n_bkg)
data_norm = normalize(data_norm, 'signal_label', n_bkg)
data_norm = normalize(data_norm, 'channel', n_bkg)
data_processed, channel_indices = bucketize(data_norm, 'channel')
print(list(data_processed.keys()))
print(channel_indices)

['event', 'genWeight', 'deltaphi_12', 'deltaphi_13', 'deltaphi_23', 'deltaeta_12', 'deltaeta_13', 'deltaeta_23', 'deltaR_12', 'deltaR_13', 'deltaR_23', 'pt_123', 'mt_12', 'mt_13', 'mt_23', 'Mt_tot', 'n_tauh', 'signal_label', 'channel', 'event_type', 'mass_hyp']
{'tte': 0, 'tee': 1, 'tmm': 2, 'tem': 3, 'ttm': 4}


In [9]:
input_vars = ['deltaphi_12', 'deltaphi_13', 'deltaphi_23', 'deltaeta_12', 'deltaeta_13', 'deltaeta_23', 'deltaR_12', 'deltaR_13', 'deltaR_23',
              'pt_123', 'mt_12', 'mt_13', 'mt_23', 'Mt_tot', 'signal_label', 'channel', 'mass_hyp']

In [10]:
train, val, test, meas = split_dataset(data_processed)
x_train = train[input_vars]
x_test = test[input_vars]
x_val = val[input_vars]
x_meas = meas[input_vars]

label_train = x_train.pop('signal_label').astype(float)
label_val = x_val.pop('signal_label').astype(float)
label_test = x_test.pop('signal_label').astype(float)
label_meas = x_meas.pop('signal_label').astype(float)

learning_features = list(x_train.keys())

Total number of events :  6818970
Train set : 37.52 %
Validation set : 12.51 %
Test set : 24.98 %
Measurement set : 24.99 %


In [ ]:
depths = [len(learning_features)*2]*5
print(depths)
model = Dnn_tau(list(x_train.keys()), depths=depths)
model.compile(loss='binary_crossentropy', 
                  optimizer='adam', 
                  metrics=['loss', 'accuracy'])

early_stopping = tf.keras.callbacks.EarlyStopping(monitor='accuracy', patience=7)
checkpoint = tf.keras.callbacks.ModelCheckpoint(
    filepath="./saved_models/checkpoint",
    monitor = "val_loss",
    save_best_only = True
)
history = model.fit(x_train, label_train, sample_weight=train['genWeight'], validation_data=(x_val, label_val), epochs=100000, verbose=1, 
                    batch_size = 400, callbacks=[early_stopping, checkpoint])
model = tf.keras.models.load_model('./saved_models/checkpoint')
model.save('./saved_models/TEST9_global_v2')
# Save history
filename = "./saved_history/TEST9_global_vs.pkl"
with open(filename, "wb") as file:
    pickle.dump(history.history, file)

[32, 32, 32, 32, 32]
Epoch 1/100000
6397/6397 [==============================] - 44s 7ms/step - loss: 1.7969 - accuracy: 0.6520 - val_loss: 0.7797 - val_accuracy: 0.7105
INFO:tensorflow:Assets written to: ./saved_models/checkpoint/assets


INFO:tensorflow:Assets written to: ./saved_models/checkpoint/assets


Epoch 2/100000
6397/6397 [==============================] - 44s 7ms/step - loss: 1.0454 - accuracy: 0.7124 - val_loss: 0.6765 - val_accuracy: 0.7170
INFO:tensorflow:Assets written to: ./saved_models/checkpoint/assets


INFO:tensorflow:Assets written to: ./saved_models/checkpoint/assets


Epoch 3/100000
6397/6397 [==============================] - 42s 7ms/step - loss: 1.0069 - accuracy: 0.7262 - val_loss: 0.6908 - val_accuracy: 0.7312
Epoch 4/100000
6397/6397 [==============================] - 43s 7ms/step - loss: 0.9830 - accuracy: 0.7347 - val_loss: 0.6897 - val_accuracy: 0.7478
Epoch 5/100000
6397/6397 [==============================] - 43s 7ms/step - loss: 0.9690 - accuracy: 0.7340 - val_loss: 0.7103 - val_accuracy: 0.7182
Epoch 6/100000
6397/6397 [==============================] - 46s 7ms/step - loss: 0.9618 - accuracy: 0.7378 - val_loss: 0.6960 - val_accuracy: 0.7354
Epoch 7/100000
6397/6397 [==============================] - 44s 7ms/step - loss: 0.9616 - accuracy: 0.7346 - val_loss: 0.7457 - val_accuracy: 0.7308
Epoch 8/100000
6397/6397 [==============================] - 43s 7ms/step - loss: 0.9527 - accuracy: 0.7386 - val_loss: 0.6268 - val_accuracy: 0.7603
INFO:tensorflow:Assets written to: ./saved_models/checkpoint/assets


INFO:tensorflow:Assets written to: ./saved_models/checkpoint/assets


Epoch 9/100000
6397/6397 [==============================] - 44s 7ms/step - loss: 0.9494 - accuracy: 0.7395 - val_loss: 0.6843 - val_accuracy: 0.7387
Epoch 10/100000
6397/6397 [==============================] - 42s 7ms/step - loss: 0.9528 - accuracy: 0.7380 - val_loss: 0.7081 - val_accuracy: 0.7268
Epoch 11/100000
6397/6397 [==============================] - 43s 7ms/step - loss: 0.9416 - accuracy: 0.7413 - val_loss: 0.6744 - val_accuracy: 0.7381
Epoch 12/100000
6397/6397 [==============================] - 42s 7ms/step - loss: 0.9351 - accuracy: 0.7445 - val_loss: 0.6708 - val_accuracy: 0.7643
Epoch 13/100000
6397/6397 [==============================] - 44s 7ms/step - loss: 0.9348 - accuracy: 0.7430 - val_loss: 0.6280 - val_accuracy: 0.7537
Epoch 14/100000
6397/6397 [==============================] - 44s 7ms/step - loss: 0.9313 - accuracy: 0.7466 - val_loss: 0.7300 - val_accuracy: 0.7131
Epoch 15/100000
6397/6397 [==============================] - 43s 7ms/step - loss: 0.9305 - accuracy: 

INFO:tensorflow:Assets written to: ./saved_models/TEST9_global_v2/assets


INFO:tensorflow:Assets written to: ./saved_models/TEST9_global_v2/assets


In [11]:
model = tf.keras.models.load_model('./saved_models/TEST9_global_v2')

2023-05-01 07:58:30.822725: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-01 07:58:31.681928: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 6673 MB memory:  -> device: 0, name: Quadro RTX 4000, pci bus id: 0000:17:00.0, compute capability: 7.5
2023-05-01 07:58:31.682766: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 6653 MB memory:  -> device: 1, name: Quadro RTX 4000, pci bus id: 0000:65:00.0, compute capability: 7.5


In [12]:
print(model.metrics_names)

[]


In [11]:
delta_loss_perm = []
for key in learning_features:
    loss_no_shuffle, loss_shuffle = fi_perm(model, test, input_vars, key)
    delta_loss_perm.append(loss_shuffle-loss_no_shuffle)
delta_loss_perm = dict(zip(learning_features, delta_loss_perm))
print(delta_loss_perm)

x_test defined
y_test defined
Loss without shuffle evaluated
shuffling of  deltaphi_12
	 column copied
	 column shuffled
	 column assigned
Loss with shuffle evaluated
x_test defined
y_test defined
Loss without shuffle evaluated
shuffling of  deltaphi_13
	 column copied
	 column shuffled
	 column assigned
Loss with shuffle evaluated
x_test defined
y_test defined
Loss without shuffle evaluated
shuffling of  deltaphi_23
	 column copied
	 column shuffled
	 column assigned
Loss with shuffle evaluated
x_test defined
y_test defined
Loss without shuffle evaluated
shuffling of  deltaeta_12
	 column copied
	 column shuffled
	 column assigned
Loss with shuffle evaluated
x_test defined
y_test defined
Loss without shuffle evaluated
shuffling of  deltaeta_13
	 column copied
	 column shuffled
	 column assigned
Loss with shuffle evaluated
x_test defined
y_test defined
Loss without shuffle evaluated
shuffling of  deltaeta_23
	 column copied
	 column shuffled
	 column assigned
Loss with shuffle evaluate

In [13]:
filename = "./saved_results/TEST9_global_v1_loss_shuffle.pkl"
with open(filename, "wb") as file:
    pickle.dump(delta_loss_perm, file)

In [3]:
filename = "./saved_results/TEST9_global_v1_loss_shuffle.pkl"
with open(filename, "rb") as f:
    delta_loss_perm = pickle.load(f)

In [13]:
x_test = test[input_vars]
print("x_test defined")
y_test = x_test.pop('signal_label').astype(float)
print("y_test defined")

loss_no_shuffle, _ = model.evaluate(x_test, y_test, sample_weight=test['genWeight'], verbose=0)
print("Loss without shuffle evaluated")

key = input_vars[0]
print(x_test[key][:20])
x_test[key] = np.random.permutation(x_test[key])
print(x_test[key][:20])

print("\t column assigned")
loss_shuffle, _ = model.evaluate(x_test, y_test, sample_weight=test['genWeight'], verbose=0)
print("Loss with shuffle evaluated")

x_test defined
y_test defined
Loss without shuffle evaluated
0     0.619110
1     0.510193
2     1.376953
3     0.476545
4     2.452375
5     2.357056
6     0.287598
7     0.482910
8     2.813477
9     0.016235
10    1.205933
11    2.167951
12    1.520935
13    3.052490
14    2.583008
15    1.210694
16    0.617126
17    1.739728
18    1.042237
19    0.391113
Name: deltaphi_12, dtype: float64
shuffling of  deltaphi_12
	 column copied
	 column shuffled
0     0.619110
1     0.510193
2     1.376953
3     0.476545
4     2.452375
5     2.357056
6     0.287598
7     0.482910
8     2.813477
9     0.016235
10    1.205933
11    2.167951
12    1.520935
13    3.052490
14    2.583008
15    1.210694
16    0.617126
17    1.739728
18    1.042237
19    0.391113
Name: deltaphi_12, dtype: float64
	 column assigned


KeyboardInterrupt: 